In [2]:
from aiida import load_profile, get_profile
profile= load_profile()
print(profile.name)
print(profile.database_name)
profile.name
profile.database_name 
get_profile()


default
default_mozumder_0077f85362e7bb995525ed5597a1b24e


In [6]:
# imports from aiida-core and aiida-kkr
from aiida.orm import CifData, Code, Dict, load_node
from aiida.engine import submit
from aiida_kkr.tools import kkrparams, plot_kkr
import os

cif = CifData(file=os.path.abspath('/opt/judft_tutorials/tutorials/iffaiida_tutorial_01_2020/aiida-kkr/data/Fe_mp-13_symmetrized.cif'))
struc = cif.get_structure()

# the ase-notebook viewer also accepts additional keyword arguments like the unit cell repitition 'repeat_uc', 'zoom' or 'canvas_size':
p = plot_kkr(struc, repeat_uc=(2,2,2), zoom=1, canvas_size=(600,200), silent=True)

GridspecLayout(children=(Box(children=(Renderer(camera=PerspectiveCamera(far=1000.0, fov=10.0, near=1.0, posit…

In [5]:
# use pymatgen to find primitive structure
prim_struc = struc.get_pymatgen_structure().get_primitive_structure()
# convert to AiiDA StructureData
from aiida.orm import StructureData
struc_prim = StructureData(pymatgen=prim_struc)

plot_kkr(struc_prim, silent=True)


GridspecLayout(children=(Renderer(camera=PerspectiveCamera(fov=10.0, position=(0.0, 0.0, 33.83687195684682), p…

In [35]:
from aiida_kkr.workflows import kkr_scf_wc
settings, _ = kkr_scf_wc.get_wf_defaults()

# activate magnetic initialization, reduce convegence criterion and increase mixing
settings['mag_init'] = True
settings['natom_in_cls_min'] = 25
settings['convergence_criterion'] = 1e-6
settings['nsteps'] = 100
settings['threshold_switch_high_accuracy'] = 1e-2

Version of workflow: 0.10.4


In [39]:
metadata_option_1 = {'max_wallclock_seconds': 36000,'resources': 
               {'tot_num_mpiprocs': 24, 'num_machines': 1},
              'custom_scheduler_commands': 
              '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
              'withmpi':
              True}

In [43]:
# create process builder
builder = kkr_scf_wc.get_builder()

# set structure input using the primitive structure
builder.structure = struc_prim

# set minimals et of KKR parameters
builder.calc_parameters = Dict(dict={'LMAX': 2, 'NSPIN': 2})

# set the workflow settings
builder.wf_parameters = Dict(dict=settings)

# set the codes and options
builder.voronoi = Code.get_from_string('voro@claix18_init')
builder.kkr = Code.get_from_string('kkr@claix18_init')
# load options for the oscar partition of iffslurm (comes with iffslurm import file) which uses a single node for up to 1h with mpi
# options_slurm_oscar = load_node('3b6c327e-d894-48ef-9f82-b45570149779')
options = Dict(dict=metadata_option_1)
builder.options = options

# set a calculation label and a description (optional)
builder.metadata.label = 'bccFe'
builder.metadata.description = 'bcc Fe imported from materials project (id=13)'

# finally submit the workflow and wait
# scf = submit(builder)
# scf = load_node('dcc3e02c-1801-44d1-827e-5f750606439b')

In [45]:
# plot_kkr(scf)


In [46]:
# create process builder
builder = scf.get_builder_restart()
# overwrite structure input and set label
builder.structure = struc
builder.metadata.label = 'bccFe_conv'

# finally submit the workflow and wait
scf_conv = submit(builder)
# 7edabda1-7f4f-4eb9-b8c5-1edb6bd7d279

In [59]:
scf_conv.uuid


'7edabda1-7f4f-4eb9-b8c5-1edb6bd7d279'

In [48]:
from aiida_kkr.workflows import kkr_dos_wc
settings = kkr_dos_wc.get_wf_defaults()
settings

Version of workflow: 0.8.0


{'nepts': 96,
 'tempr': 200.0,
 'emin': -10.0,
 'emax': 5.0,
 'kmesh': [30, 30, 30],
 'RCLUSTZ': None}

In [49]:
builder = kkr_dos_wc.get_builder()
# reuse the KKRhost code and options from the scf calculation (primitive structure)
builder.kkr = scf.inputs.kkr
builder.options = scf.inputs.options
# set to converged output
builder.remote_data = scf.outputs.last_RemoteData
# set dos parameters
builder.wf_parameters = Dict(dict=settings)

dos = submit(builder)


In [54]:
from aiida_kkr.workflows import kkr_bs_wc
settings = kkr_bs_wc.get_wf_defaults()
print("'RCLUSTZ' from converged calc: {} a_lat".format(scf.outputs.last_InputParameters['RCLUSTZ']))
settings['RCLUSTZ'] = 2.3 # increase 
settings

Version of the kkr_bs_wc workflow: 0.1.1
'RCLUSTZ' from converged calc: 1.6331509900882 a_lat


{'emin': -10.0, 'emax': 5.0, 'nepts': 96, 'RCLUSTZ': 2.3, 'tempr': 50.0}

In [56]:
def get_builder_bs(scf_node, settings):
    """
    helper function to set up the process builder
    """
    builder = kkr_bs_wc.get_builder()
    # reuse the KKRhost code and options from the scf calculation
    builder.kkr = scf_node.inputs.kkr
    builder.options = scf_node.inputs.options
    # set to converged output
    builder.remote_data = scf_node.outputs.last_RemoteData
    # set dos parameters
    builder.wf_parameters = Dict(dict=settings)
    
    return builder

if scf.is_finished_ok:
    # first we submit the calculation for the primitive structure
    builder = get_builder_bs(scf, settings)
    bandstruc = submit(builder)


In [57]:
if scf_conv.is_finished_ok:
    builder = get_builder_bs(scf_conv, settings)
    #bandstruc_conv = submit(builder)